In [1]:
import pandas as pd
from datetime import datetime,timezone
from scipy.spatial.distance import pdist, squareform
from scipy.fftpack import fft
from sklearn.manifold import MDS
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import numpy as np

In [2]:
df = pd.read_csv("malekoff.csv")

In [3]:
df['time'] = pd.to_datetime(df['time'], unit = "ms")
timeA = df['time'].dt.strftime('%m-%d-%Y %H.%M')
df.insert(1, "timeA", timeA)

df.loc[:, (df.columns != 'timeA')] = df.loc[:, df.columns != 'timeA'].apply(pd.to_numeric, errors = 'coerce')
df = df.set_index('timeA')

df.drop(df.columns[[0,1]], axis=1, inplace=True)

In [4]:
df.head()

,MAL_TBP.U30.30HFE10AA001XQ01.out,MAL_TBP.U30.30HFE50AA001XQ01.out,MAL_TBP.U30.30CJA10DT109.out,MAL_TBP.U30.30HHY11CR002XQ01.out
timeA,,,,
12-26-2022 18.32,50.03125,49.75000,26.353168,0.0
12-26-2022 18.33,49.68750,49.43750,26.402964,0.0
12-26-2022 18.34,49.84375,49.50000,26.103868,0.0
12-26-2022 18.36,49.87500,49.84375,26.358898,0.0
12-26-2022 18.38,49.96875,49.96875,26.358898,0.0


In [50]:
df

,MAL_TBP.U30.30HFE10AA001XQ01.out,MAL_TBP.U30.30HFE50AA001XQ01.out,MAL_TBP.U30.30CJA10DT109.out,MAL_TBP.U30.30HHY11CR002XQ01.out
timeA,,,,
12-26-2022 18.32,50.03125,49.75000,26.353168,0.0
12-26-2022 18.33,49.68750,49.43750,26.402964,0.0
12-26-2022 18.34,49.84375,49.50000,26.103868,0.0
12-26-2022 18.36,49.87500,49.84375,26.358898,0.0
12-26-2022 18.38,49.96875,49.96875,26.358898,0.0
...,...,...,...,...
02-15-2023 06.04,0.62500,0.03125,0.446966,0.0
02-15-2023 10.21,0.62500,0.03125,0.446966,0.0
02-16-2023 00.20,0.62500,0.03125,0.446966,0.0


In [51]:
df.loc[:, (df.columns != 'timeA')] = df.loc[:, df.columns != 'timeA'].interpolate(method = 'linear')
(df.isna().sum() / df.shape[0]) * 100


MAL_TBP.U30.30HFE10AA001XQ01.out    0.0
MAL_TBP.U30.30HFE50AA001XQ01.out    0.0
MAL_TBP.U30.30CJA10DT109.out        0.0
MAL_TBP.U30.30HHY11CR002XQ01.out    0.0
dtype: float64

In [52]:
feature = df.columns[0] # "MAL_TBP.U30.30HFE10AA001XQ01.out "
sampling_period = 4*60 # in seconds of the dataset as provided
time_period =  12*60*60 # time duration in seconds corresoponding to each sample, in this case 12 hours
scaler = {'identity': FunctionTransformer(lambda x: x), 'standard': StandardScaler()}


In [53]:
df_transformed = pd.DataFrame(scaler['identity'].fit_transform(df))


In [54]:
df_transformed.index = df.index.copy()
df_transformed.columns = df.columns.copy()


In [55]:


type(df_transformed.index)
df_transformed.index = pd.to_datetime(df_transformed.index, errors='coerce')

In [56]:
df_transformed.head()

,MAL_TBP.U30.30HFE10AA001XQ01.out,MAL_TBP.U30.30HFE50AA001XQ01.out,MAL_TBP.U30.30CJA10DT109.out,MAL_TBP.U30.30HHY11CR002XQ01.out
timeA,,,,
NaT,50.03125,49.75000,26.353168,0.0
NaT,49.68750,49.43750,26.402964,0.0
NaT,49.84375,49.50000,26.103868,0.0
NaT,49.87500,49.84375,26.358898,0.0
NaT,49.96875,49.96875,26.358898,0.0


In [57]:
## Data preparation for autoregression
feature = df.columns[0] # "Cyclone_Inlet_Gas_Temp"
# Sampling period of the dataset in seconds
sampling_period = 5*60
# Lag in seconds for autoregression (in this case, we regress on the past 12 hours):
# change this for last 6 hours, 2 hours, 0.5 hours etc
lag = 12
lag = lag*60*60
scaler = {'identity': FunctionTransformer(lambda x: x), 'standard': StandardScaler()}
df_transformed = pd.DataFrame(scaler['identity'].fit_transform(df))
df_transformed.columns = df.columns.copy()
df_transformed = df_transformed.reset_index().drop(columns = ['time'])
# Number of columns and rows in the dataframe for performing autoregression
nCols = int(pd.Timedelta(str(time_period/sampling_period)+'S').total_seconds())
nRows = int(df_transformed.shape[0]/nCols)-1
# Create dataframe for autoregression
df_autoreg = pd.DataFrame(index = np.arange(nRows), columns = np.arange(nCols+1))
for j in range(nRows):
  df_autoreg.iloc[j, :] = pd.DataFrame([df_transformed.loc[j, feature]] + df_transformed.loc[j+1:j+nCols, feature].to_list()).T
df_autoreg.head()

AttributeError: 'NaTType' object has no attribute 'normalize'

: 